## Import cuxfilter

In [ ]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import *

from bokeh.tile_providers import get_provider as gp
tile_provider = gp('STAMEN_TONER')

In [ ]:
#update data_dir if you have downloaded datasets elsewhere
DATA_DIR = './data'

## Download required datasets

In [ ]:
! curl https://s3.us-east-2.amazonaws.com/rapidsai-data/viz-data/146M_predictions_v2.arrow.gz --create-dirs -o $DATA_DIR/146M_predictions_v2.arrow.gz

In [ ]:
from cuxfilter.sampledata import datasets_check
datasets_check('mortgage', base_dir=DATA_DIR)

## Define charts

In [ ]:
cux_df = cuxfilter.DataFrame.from_arrow(DATA_DIR + '/146M_predictions_v2.arrow')

In [ ]:
cux_df.data.head()

In [ ]:
MAPBOX_API_KEY= "<mapbox-api-key>"
geoJSONSource='https://raw.githubusercontent.com/rapidsai/cuxfilter/GTC-2018-mortgage-visualization/javascript/demos/GTC%20demo/src/data/zip3-ms-rhs-lessprops.json'


In [ ]:
chart0 = cuxfilter.charts.deckgl.choropleth3d(
    x='zip',
    
    color_column='delinquency_12_prediction', color_aggregate_fn='mean',
    
    elevation_column='current_actual_upb', elevation_factor=0.00001, elevation_aggregate_fn='sum', 
    
    geoJSONSource=geoJSONSource,
    
    map_style = 'dark', mapbox_api_key=MAPBOX_API_KEY,
   
    data_points=1000, tooltip=True
)

chart2 = cuxfilter.charts.bokeh.bar('delinquency_12_prediction',data_points=50)
chart3 = cuxfilter.charts.panel_widgets.range_slider('borrower_credit_score',data_points=50)


### Add mappings for chart1

In [ ]:
mapper1 = {}

for val in cux_df.data.dti.unique().to_pandas().tolist():
    mapper1[int(val)] = 'l_'+str(val)

chart1 = cuxfilter.charts.panel_widgets.drop_down('dti', label_map=mapper1)
#also available: panel_widgets.multi_select

## Create a dashboard object

In [ ]:
d = cux_df.dashboard([chart0, chart1], layout=feature_and_base, theme=cuxfilter.themes.dark, title="Mortgage Dashboard")

In [ ]:
#add more charts
d.add_charts([chart2, chart3])

## View Individual charts

In [ ]:
cuxfilter.load_notebook_assets()

In [ ]:
chart0

## Starting the dashboard

1. d.show('current_notebook_url:current_notebook_port') remote dashboard

2. d.app('current_notebook_url:current_notebook_port') inline in notebook (layout is ignored, and charts are displayed one below another)

Incase you need to stop the server:

- d.stop()

In [ ]:
d.show(notebook_url='http://localhost:8888')

## Export the queried data into a dataframe

In [ ]:
queried_df = d.export()